In [62]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob as gb
import numpy as np


In [63]:
path = os.getcwd()+ '/archive/car_data/car_data'

In [64]:
train = path+"/train"
test = path+"/test"
train_labels = []
train_images = []
test_labels = []
test_images = []


In [65]:
train_path =  [folder for folder in os.listdir(train) if not folder.startswith('.')]
test_path =  [folder for folder in os.listdir(test) if not folder.startswith('.')]


## get # of photos for train & test
train_len = -1
for i in os.walk(train, topdown=True):
        train_len += len(i[2]) 

test_len = -1
for i in os.walk(test, topdown=True):
        test_len += len(i[2]) 

## add the type of cars into dict
d = dict()
idx = 0
for folder in test_path:
    d[folder] = idx
    idx+= 1

In [66]:
# Get image annotations
# Heading names: img_name, bbox_x1, bbox_y1, bbox_x2, bbox_y2, class
train_annot = pd.read_csv(os.getcwd() + "/archive/anno_train.csv")
test_annot = pd.read_csv(os.getcwd() + "/archive/anno_test.csv")

train_bboxes = []
test_bboxes = []

train_annot.keys()

Index(['img_name', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class'], dtype='object')

In [67]:
for folder in test_path:
    jpg_path = gb.glob(pathname = test + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)

        # Get bounding box value for image
        bbox_x1 = test_annot.loc[test_annot['img_name'] == img_name]['bbox_x1']
        bbox_y1 = test_annot.loc[test_annot['img_name'] == img_name]['bbox_y1']
        bbox_x2 = test_annot.loc[test_annot['img_name'] == img_name]['bbox_x2']
        bbox_y2 = test_annot.loc[test_annot['img_name'] == img_name]['bbox_y2']

        # Append bounding box value to array
        test_bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        
        image = cv2.imread(img)
        test_labels.append(np.array(d[folder]))
        image = cv2.resize(image, (224,224))
        test_images.append(np.array(image))

for folder in train_path:
    jpg_path = gb.glob(pathname = train + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)

        # Get bounding box value for image
        bbox_x1 = train_annot.loc[train_annot['img_name'] == img_name]['bbox_x1']
        bbox_y1 = train_annot.loc[train_annot['img_name'] == img_name]['bbox_y1']
        bbox_x2 = train_annot.loc[train_annot['img_name'] == img_name]['bbox_x2']
        bbox_y2 = train_annot.loc[train_annot['img_name'] == img_name]['bbox_y2']
        
        # Append bounding box value to array
        train_bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))

        image = cv2.imread(img)
        train_labels.append(np.array(d[folder]))
        image = cv2.resize(image, (224,224))
        train_images.append(np.array(image))
        

In [68]:
# Scale bounding box to image resolution

for i in range(0, len(train_bboxes)):
    for j in range(0,4):
        train_bboxes[i][j] / 224

for i in range(0, len(test_bboxes)):
    for j in range(0,4):
        test_bboxes[i][j] / 224

# Convert from array of pandas series to floats
train_bboxes = np.array(train_bboxes, dtype="float64")
test_bboxes = np.array(test_bboxes, dtype="float64")


In [69]:
train_bboxes.shape

(131, 4, 1)

In [70]:
train_labels = np.array(train_labels)
train_labels = np.reshape(train_labels, (-1, 1))

test_labels = np.array(test_labels)
test_labels = np.reshape(test_labels, (-1, 1))

train_images = np.array(train_images)
test_images = np.array(test_images)


In [71]:
print(test_labels)
print(test_labels.shape)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]]
(129, 1)


In [72]:
train_images, test_images= train_images / 255.0, test_images/ 255.0


In [73]:
from keras import layers
from keras.models import Model
from keras.applications import VGG16
model = VGG16(weights="imagenet", include_top=False, input_tensor=layers.Input(shape=(224, 224, 3)))

model.trainable=True

flatten = model.output
flatten = layers.Flatten() (flatten)

"""
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
"""

"\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(224, 224, 3)))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))\n"

In [74]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [75]:
# Fully connected layer header for bounding boxes
bbox_output = layers.Dense(128, activation='relu') (flatten)
bbox_output = layers.Dense(64, activation='relu') (bbox_output)
bbox_output = layers.Dense(32, activation='relu') (bbox_output)
bbox_output = layers.Dense(1, activation="sigmoid", name="bbox") (bbox_output)

In [76]:
# Second fully connected layer header for predicting class label
label_output = layers.Dense(512, activation="relu")(flatten)
label_output = layers.Dropout(0.5) (label_output)
label_output = layers.Dense(512, activation="relu") (label_output)
label_output = layers.Dropout(0.5) (label_output)
label_output = layers.Dense(3, activation="softmax", name="class_label") (label_output)

model = Model(inputs=model.input, outputs=(bbox_output, label_output))
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [77]:
for folder in train_path:
    jpg_path = gb.glob(pathname = test + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)
        image = cv2.imread(img)
        np.append(test_labels,np.array(folder))
        image = cv2.resize(image, (224,224))
        np.append(test_images,np.array(image))

In [78]:
# Set losses for both fully-connected layer headers
from keras.losses import SparseCategoricalCrossentropy

losses = {
    "class_label": SparseCategoricalCrossentropy(from_logits=False),
    "bbox": "mean_squared_error"
}

lossWeights = {
    "class_label": 1.0,
	"bbox": 1.0
}

# Set targets
train_targets = {
    "class_label": train_labels,
    "bbox": train_bboxes
}

test_targets = {
    "class_label": test_labels,
    "bbox": test_bboxes
}

In [79]:
model.compile(optimizer='adam',
              loss=losses, loss_weights=lossWeights,
              metrics=['accuracy'])

history = model.fit(train_images, train_targets, epochs=10,
                    validation_data=(test_images, test_targets), batch_size=32, verbose=1)

Epoch 1/10
5/5 [==============================] - 68s 13s/step - loss: 185413.6719 - bbox_loss: 185383.0000 - class_label_loss: 30.6870 - bbox_accuracy: 0.0095 - class_label_accuracy: 0.3130 - val_loss: 182495.2812 - val_bbox_loss: 182494.1719 - val_class_label_loss: 1.1070 - val_bbox_accuracy: 0.0116 - val_class_label_accuracy: 0.3101
Epoch 2/10
5/5 [==============================] - 63s 13s/step - loss: 185320.1406 - bbox_loss: 185316.6406 - class_label_loss: 3.4845 - bbox_accuracy: 0.0115 - class_label_accuracy: 0.2519 - val_loss: 182495.6562 - val_bbox_loss: 182494.2500 - val_class_label_loss: 1.4167 - val_bbox_accuracy: 0.0116 - val_class_label_accuracy: 0.3411
Epoch 3/10
5/5 [==============================] - 63s 13s/step - loss: 185318.5469 - bbox_loss: 185316.7188 - class_label_loss: 1.8127 - bbox_accuracy: 0.0115 - class_label_accuracy: 0.3817 - val_loss: 182495.3594 - val_bbox_loss: 182494.1406 - val_class_label_loss: 1.2349 - val_bbox_accuracy: 0.0116 - val_class_label_accur

model.summary()

In [ ]:
y_pred = model.predict(test_images)
pred_label = np.argmax(y_pred,axis = 1)
plt.figure(figsize=(25,25))

def getLabel(i):
    return (list(d.keys())[list(d.values()).index(i)])  

for i in range(25):
    
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    plt.imshow(test_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.title("True: " + getLabel(test_labels[i][0]) + "\n" + "Predicted: " + getLabel(pred_label[i]))
plt.tight_layout()
plt.show()